In [ ]:
!rm -Rf HMP_Dataset
!git clone https://github.com/wchill/HMP_Dataset

In [ ]:
!ls HMP_Dataset/Brush_teeth

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType

schema = StructType([
    StructField("x", IntegerType(), True),
    StructField("y", IntegerType(), True),
    StructField("z", IntegerType(), True)])

In [ ]:
import os

#get list of folders/files in folder HMP_Dataset
file_list = os.listdir('HMP_Dataset')

#filter list for folders containing data
file_list_filtered = [s for s in file_list if '_' in s]

In [ ]:
import os

#get list of folders/files in folder HMP_Dataset
file_list = os.listdir('HMP_Dataset')

#filter list for folders containing data
file_list_filtered = [s for s in file_list if '_' in s]

from pyspark.sql.functions import lit

#create pandas data frame for all the data

df = None

for category in file_list_filtered:
    data_files = os.listdir('HMP_Dataset/'+category)
    
    #create a temporary pandas data frame for each data file
    for data_file in data_files:
        print(data_file)
        temp_df = spark.read.option("header", "true").option("header", "false").option("delimiter", " ").csv('HMP_Dataset/'+category+'/'+data_file,schema=schema)
        
        #create a column called "source" storing the current CSV file
        temp_df = temp_df.withColumn("source", lit(data_file))
        
        #create a column called "class" storing the current data folder
        temp_df = temp_df.withColumn("class", lit(category))
        
        #append to existing data frame list
        #data_frames = data_frames + [temp_df]
                                                                                                             
        if df is None:
            df = temp_df
        else:
            df = df.union(temp_df)
        


In [ ]:

import ibmos2spark
# @hidden_cell
credentials = {
    'endpoint': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'service_id': 'iam-ServiceId-d4b06e46-293a-4417-b76c-2f16076a9353',
    'iam_service_endpoint': 'https://iam.bluemix.net/oidc/token',
    'api_key': 'iTfSE_YVE6zqnxjd1oK0E37R2aVYY4dFhXFFCljE1AcJ'
}

configuration_name = 'os_b0f1407510994fd1b793b85137baafb8_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')


# please put a file name here since we are usinf a shared objectstore, e.g. hmp_your_name.parquet
df.write.parquet(cos.url(###_YOUR_FILENAME_GOES_HERE###, 'courseraml-donotdelete-pr-qve0ttzezdeodc'))
